# Hello
My good friend created this code.  

In [ ]:
import numpy as np
from scipy.optimize import minimize

def squared_exponential_kernel(x1, x2, sigma_s, W):
    """Squared Exponential Kernel function."""
    diff = x1 - x2
    return sigma_s * np.exp(-0.5 * diff.T @ W @ diff)

def compute_kernel_matrix(X, sigma_s, W, sigma_n):
    """Compute the kernel matrix with added noise."""
    n = X.shape[0]
    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i, j] = squared_exponential_kernel(X[i], X[j], sigma_s, W)
    K += sigma_n * np.eye(n)  # Add noise variance
    return K

def negative_log_likelihood(theta, X, y):
    """Negative log-likelihood for GP."""
    sigma_s, sigma_n = np.exp(theta[:2])  # Exponentiate to ensure positivity
    W = np.diag(np.exp(theta[2:]))  # Diagonal matrix for weights
    K = compute_kernel_matrix(X, sigma_s, W, sigma_n)
    L = np.linalg.cholesky(K)  # Cholesky decomposition for numerical stability
    alpha = np.linalg.solve(L.T, np.linalg.solve(L, y))
    
    log_likelihood = -0.5 * y.T @ alpha - np.sum(np.log(np.diagonal(L))) - (X.shape[0] / 2) * np.log(2 * np.pi)
    return -log_likelihood  # Return negative for minimization

# Example usage
# Generate synthetic data
np.random.seed(0)
X = np.random.uniform(-5, 5, (20, 1))  # Input data
y = np.sin(X).flatten() + 0.1 * np.random.randn(X.shape[0])  # Noisy outputs

# Initial hyperparameters: log(sigma_s), log(sigma_n), log(diag(W))
initial_theta = np.log([1.0, 0.1] + [1.0] * X.shape[1])

# Minimize negative log-likelihood
result = minimize(negative_log_likelihood, initial_theta, args=(X, y), method="L-BFGS-B")

# Extract optimal hyperparameters
opt_theta = result.x
sigma_s_opt, sigma_n_opt = np.exp(opt_theta[:2])
W_opt = np.diag(np.exp(opt_theta[2:]))
print("Optimal Hyperparameters:")
print(f"sigma_s: {sigma_s_opt}, sigma_n: {sigma_n_opt}, W: {W_opt}")

ValueError: operands could not be broadcast together with shapes (80,) (20,) 